In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm_model = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm = llm_model,
    max_token_limit=120,
    # memory_key의 기본 값 -> "history", 따라서 chat_history 대신 history 사용 가능
    memory_key = "chat_history",
    return_messages = True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name = "chat_history"),
        ("human", "{question}")
    ]
)

def load_memory(input):
    # print(input)
    return memory.load_memory_variables({})["chat_history"]

# load_memory 리턴값이 chat_history에 저장됨
# chat_history와 사용자의 input이 결합된 후 프롬프트에
# runnablepassthrough를 통해 전달됨
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm_model

def invoke_chain(question):
    result = chain.invoke({
    # 체인 호출마다 chat_history를 추가해줘야함
    # "chat_history": memory.load_memory_variables({})["chat_history"],
    # 체인의 첫번째 요소의 input이 됨
    "question" : question
    })
    memory.save_context({"input": question}, {"output": result.content},)

    print(result)

In [6]:
invoke_chain("My name is Charls.")

Failed to load default session, using empty session: 0
Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


content='Hello Charls! How can I assist you today?'


In [7]:
invoke_chain("what is my name?")

Failed to load default session, using empty session: 0
Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


content='Your name is Charls.'
